In [37]:
import json
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import multinomial
from scipy.stats import kstest, chisquare
from collections import defaultdict

In [38]:
exp_name = "lt-at-al-part2"

In [39]:
exp = np.array(exp_name.split("-")[:3])
string_answer_to_code = {
    'All are Same Species': "1, 1, 1",
    'ONLY 1 and 2 are Same Species': "1, 1, 0",
    'ONLY 2 and 3 are Same Species': "0, 1, 1",
    'ONLY 1 and 3 are Same Species': "1, 0, 1",
    'NONE, all are different species': "0, 1, 2"
}

In [40]:
with open(f"data/{exp_name}.json") as fp:
    data = json.load(fp)

In [41]:
confusion_matrix = defaultdict(lambda: defaultdict(int))

for worker in data:
    for answer in worker['answer']:
        if 'originalOrder' not in answer or len(answer['originalOrder']) == 0:
            continue
        args = answer['args']
        question_order = tuple(exp[args])
        if sum(confusion_matrix[", ".join(question_order)].values()) == 980:
            continue
        question_answer = string_answer_to_code[answer['clusters']]
        confusion_matrix[", ".join(question_order)][question_answer]  += 1
            

In [42]:
count_df = pd.DataFrame(confusion_matrix).fillna(0)
count_df

,"lt, al, at","at, lt, al","al, at, lt","al, lt, at","lt, at, al","at, al, lt"
"0, 1, 2",264,279,279,262,279,279
"1, 0, 1",365,168,183,169,173,367
"1, 1, 0",146,342,109,129,357,117
"0, 1, 1",149,133,344,366,120,165
"1, 1, 1",56,58,65,54,51,52


In [43]:
count_df.sum()

lt, al, at    980
at, lt, al    980
al, at, lt    980
al, lt, at    980
lt, at, al    980
at, al, lt    980
dtype: int64

In [44]:
def get_unified_pmf(col, count_df):
    mapping = {
        '1, 1, 1': 0,
        '1, 1, 0': 1,
        '1, 0, 1': 2,
        '0, 1, 1': 3,
        '0, 1, 2': 4,
        '2, 0, 1': 4,
        '2, 1, 0': 4,
        '1, 2, 0': 4,
        '1, 0, 2': 4,
        '0, 2, 1': 4,
    }
    pmf = np.zeros(5)
    arg_idx = np.argsort(col.split(', '))
    for idx, n in count_df[col].items():
        new_idx = ', '.join(np.array(idx.split(', '))[arg_idx])
        pmf[mapping[new_idx]] = n
    return pmf

In [45]:
np.random.seed(1)
for exp_i, exp_j in combinations(count_df.columns, 2):
    f_i = get_unified_pmf(exp_i, count_df)
    f_j = get_unified_pmf(exp_j, count_df)
    result = chisquare(f_obs=f_i, f_exp=f_j)
    print(f'{exp_i} vs. {exp_j}: statistic: {result.statistic:.3f} p-value: {result.pvalue:.3f} corrected: {result.pvalue * 15:.3f}', f_i, f_j)

lt, al, at vs. at, lt, al: statistic: 5.842 p-value: 0.211 corrected: 3.169 [ 56. 149. 146. 365. 264.] [ 58. 168. 133. 342. 279.]
lt, al, at vs. al, at, lt: statistic: 25.494 p-value: 0.000 corrected: 0.001 [ 56. 149. 146. 365. 264.] [ 65. 109. 183. 344. 279.]
lt, al, at vs. al, lt, at: statistic: 4.699 p-value: 0.320 corrected: 4.794 [ 56. 149. 146. 365. 264.] [ 54. 169. 129. 366. 262.]
lt, al, at vs. lt, at, al: statistic: 12.698 p-value: 0.013 corrected: 0.193 [ 56. 149. 146. 365. 264.] [ 51. 120. 173. 357. 279.]
lt, al, at vs. at, al, lt: statistic: 12.065 p-value: 0.017 corrected: 0.253 [ 56. 149. 146. 365. 264.] [ 52. 117. 165. 367. 279.]
at, lt, al vs. al, at, lt: statistic: 46.362 p-value: 0.000 corrected: 0.000 [ 58. 168. 133. 342. 279.] [ 65. 109. 183. 344. 279.]
at, lt, al vs. al, lt, at: statistic: 3.103 p-value: 0.541 corrected: 8.111 [ 58. 168. 133. 342. 279.] [ 54. 169. 129. 366. 262.]
at, lt, al vs. lt, at, al: statistic: 30.040 p-value: 0.000 corrected: 0.000 [ 58. 168